In [1]:
library(biomaRt)
library(dplyr)
library(readr)


Attaching package: ‘dplyr’


The following object is masked from ‘package:biomaRt’:

    select


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
humanlist <- c("acp2","calsyntenin-1","calsyntenin-2","calsyntenin-3","aplp1","aplp2","app","axl","bcma","betacellulin","betaglycan","car","cd147","cd200","cd43","cd44","cd99","cx3cl1","cxcl16","desmoglein-2","dner","dr6","dystroglycan","e-cadherin","epcam","epha2","epha5","epha7","ephb3","ephb4","ephb6","ephrin-b2","erbb4","f11r","fgfr4","glg1","hla-a2","ifnar2","igf-1r","il11r","il-1r1","il-1r2","il6r","ir","irela","ilrelb","jagged2","kcne2","kcne1","klotho","lar","ldlr","lrp1","lrp1b","lrp6","mer","met","muc1","musk","n-cadherin","neogenin","neurexin-3-b","NLRR3","p75NTR","pianp","plxdc2","pmel17","pdpn","polycystin-1","polyductin","protocadherin-12","ptk7","rage","robo1","sez6","sez6l","sez6l2","sirpa","sorcs1","sorla","sortilin","synedecan-1","synedecan-2","synedecan-3","tie1","tmeff2","tnfr1","trem2","trka","tyro3","vasorin","vegfr3")

In [3]:
mouselist <- c("apoER2","CACHD1","CADM1","CSF1R","Delta1","dscam","dscaml1","ephb2","ephrin-b2","fgfr3","ghr","l1","nectin1a","nectin3","ng2","notch1","notch2","notch3","notch4","nprc","nradd","pianp","prima","pvrl2","sez6","tkrB","trop2","tyrp1","tyrp2","vgscb1","vgscb2","vgscb3","vgscb4","vldlr")

In [9]:
mouse_mart <- useEnsembl("ensembl", dataset = "mmusculus_gene_ensembl",host = "useast.ensembl.org")

In [10]:
# Get UniProt IDs for these proteins
uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "external_gene_name",
                      values = mouselist,
                      mart = mouse_mart)


In [11]:
mouse_ids <- uniprot_mouse$uniprotswissprot

In [12]:
uniprot_mouse <- getBM(attributes = c("uniprotswissprot", "mgi_symbol"),
                      filters = "uniprotswissprot",
                      values = mouse_ids,
                      mart = mouse_mart)


In [13]:
mouse_genes <- uniprot_mouse$mgi_symbol

In [14]:
mapping <- read.delim("HMD_HumanPhenotype.rpt", header = FALSE)

In [15]:
colnames(mapping) <- c("Human_Gene","hum_num", "Mouse_Gene", "Mouse_MGI_ID", "HPO_Term","last")

In [16]:
result <- mapping %>%
  filter(Mouse_Gene %in% mouse_genes) %>%
  select(Mouse_Gene, Human_Gene)

In [17]:
converted_mouse <- result$Human_Gene

In [18]:
ensembl <- useEnsembl("ensembl", dataset = "hsapiens_gene_ensembl", host = "https://useast.ensembl.org")


In [19]:
# Fetches hgnc_symbol and unirpot ID
results_hum <- getBM(
  attributes = c("hgnc_symbol","uniprotswissprot"), # Attributes: UniProt ID and Gene Symbol
  filters = "hgnc_symbol",                      # Filter by UniProt IDs
  values = humanlist,                                 # Input your list of UniProt IDs
  mart = ensembl                                      # Use the Ensembl mart connection
)

In [21]:
human_genes <- results_hum$hgnc_symbol

In [23]:
human_genes <-c(human_genes,mouse_genes)

In [25]:
# Retriving module 1 from mic-3 since its the only microglia cell state that assigned PSEN1 to a module

In [26]:
modules <- read.csv("microstate3_modules.csv")
modules <- subset(modules, module == "micro1")

In [27]:
# Extracting the genes assigned to the same module PSEN1 is in

,gene_name,module,color,kME_grey,kME_micro1,kME_micro2,kME_micro3,kME_micro4,kME_micro5,kME_micro6
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AL669831.5,micro1,turquoise,0.21548615,0.19759714,0.16136541,0.12941490,0.17119379,0.145077531,0.046423244
3,INTS11,micro1,turquoise,0.09696439,0.09533079,0.06297456,0.06963743,0.06518315,0.094385586,0.066102929
5,SLC35E2B,micro1,turquoise,0.12806585,0.12348601,0.09901677,0.05368261,0.11732960,0.067810146,0.027891017
7,SLC35E2A,micro1,turquoise,0.09733040,0.12928621,0.05753553,0.04001120,0.08556635,0.032732457,-0.011421935
8,NADK,micro1,turquoise,0.11942740,0.16027853,0.05077375,0.04094430,0.09030611,0.124785534,0.022551881
10,CFAP74,micro1,turquoise,0.07441513,0.08557692,0.05595365,-0.01330175,0.09994148,0.001779519,-0.029470397
13,MORN1,micro1,turquoise,0.05722561,0.06019965,0.04808820,0.02155694,0.08230194,0.032214528,-0.007509752
15,TNFRSF14,micro1,turquoise,0.11220602,0.11903512,0.07703686,0.05483505,0.08161141,0.094138670,0.089482045
16,CEP104,micro1,turquoise,0.10945018,0.09528290,0.08672772,0.06460826,0.11559400,0.029898061,0.048579051


In [ ]:
module_genes <- modules$gene_name

In [ ]:
modules_genes

In [30]:
# Finding the intersection

In [38]:
intersect_genes <- intersect(human_genes,module_genes)

In [39]:
intersect_genes

[1] "CD99"   "PLXDC2" "GLG1"   "AXL"    "SORCS1" "CXCL16" "ACP2"   "IL6R"  
[9] "F11R"